# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [4]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(df['homeowner'])
df["homeowner"] = label_encoder.transform(df["homeowner"])
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,1,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,1,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,2,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,1,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,0,53000,0.433962,5,1,23000,low_risk


In [6]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df["loan_status"]

In [7]:
X.describe()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,0.606144,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,0.667811,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,0.000000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,0.000000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,1.000000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,1.000000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,2.000000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [9]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [10]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(58152, 8)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [11]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()

In [12]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = data_scaler.fit(X_train)

In [13]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [15]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9543211898288821

In [16]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  571,    54],
       [   93, 18666]])

In [17]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [20]:
# Resample the training data with the RandomOversampler

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled_ros, y_resampled_ros = ros.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled_ros)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [21]:
# Train the Logistic Regression model using the resampled data
model_ros = LogisticRegression(solver='lbfgs', random_state=1)
model_ros.fit(X_resampled_ros, y_resampled_ros)

LogisticRegression(random_state=1)

In [22]:
# Calculated the balanced accuracy score
y_pred_ros = model_ros.predict(X_test_scaled)
bas_ros = balanced_accuracy_score(y_test, y_pred_ros)
bas_ros

0.9946414201183431

In [23]:
# Display the confusion matrix
cm_ros = confusion_matrix(y_test, y_pred_ros)
cm_ros_df = pd.DataFrame(
    cm_ros, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_ros_df

,Predicted High Risk,Predicted Low Risk
Actual High Risk,622,3
Actual Low Risk,111,18648


In [24]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_ros))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [26]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

# View the count of target classes with Counter
X_resampled_s, y_resampled_s = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train)

In [27]:
# Train the Logistic Regression model using the resampled data
model_s = LogisticRegression(solver='lbfgs', random_state=1)
model_s.fit(X_resampled_s, y_resampled_s)

LogisticRegression(random_state=1)

In [28]:
# Calculated the balanced accuracy score
y_pred_s = model_s.predict(X_test_scaled)
bas_s = balanced_accuracy_score(y_test, y_pred_s)
bas_s

0.9946414201183431

In [29]:
# Display the confusion matrix
cm_s = confusion_matrix(y_test, y_pred_s)
cm_s_df = pd.DataFrame(
    cm_s, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_s_df

,Predicted High Risk,Predicted Low Risk
Actual High Risk,622,3
Actual Low Risk,111,18648


In [30]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_s))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [31]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled_cc, y_resampled_cc = cc.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled_cc)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [32]:
# Train the Logistic Regression model using the resampled data
model_cc = LogisticRegression(solver='lbfgs', random_state=1)
model_cc.fit(X_resampled_cc, y_resampled_cc)

LogisticRegression(random_state=1)

In [33]:
# Calculate the balanced accuracy score
y_pred_cc = model_cc.predict(X_test_scaled)
bas_cc =balanced_accuracy_score(y_test, y_pred_cc)
bas_cc

0.9932813049736127

In [34]:
# Display the confusion matrix
cm_cc = confusion_matrix(y_test, y_pred_cc)
cm_cc_df = pd.DataFrame(cm_cc, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_cc_df

,Predicted High Risk,Predicted Low Risk
Actual High Risk,620,5
Actual Low Risk,102,18657


In [35]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_cc))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.99      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18759

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [36]:
# Resample the training data with SMOTEENN

from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled_smoteenn, y_resampled_smoteenn= smote_enn.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled_smoteenn)

Counter({'high_risk': 55443, 'low_risk': 55906})

In [37]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model_smoteenn = LogisticRegression(solver='lbfgs', random_state=1)
model_smoteenn.fit(X_resampled_smoteenn, y_resampled_smoteenn)

LogisticRegression(random_state=1)

In [38]:
# Calculate the balanced accuracy score
y_pred_smoteenn = model_smoteenn.predict(X_test_scaled)
bas_smoteenn = balanced_accuracy_score(y_test, y_pred_smoteenn)
bas_smoteenn

0.9946414201183431

In [39]:
# Display the confusion matrix
cm_smoteenn = confusion_matrix(y_test, y_pred_smoteenn)
cm_smoteenn_df = pd.DataFrame(cm_smoteenn, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_smoteenn_df

,Predicted High Risk,Predicted Low Risk
Actual High Risk,622,3
Actual Low Risk,111,18648


In [40]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_smoteenn))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.


In [44]:
print(f'1.Naive Random Sampling, SMOTE Oversampling and Comnination Sampling (SMOTEENN) have the same balanced accuracy score of {bas_ros}, which is higher than the balanced accuracy scores runned in Simple Logistic Regression of {bas_s} and Undersampling of {bas_cc}')


1.Naive Random Sampling, SMOTE Oversampling and Comnination Sampling (SMOTEENN) have the same balanced accuracy score of 0.9946414201183431, which is higher than the balanced accuracy scores runned in Simple Logistic Regression of 0.9946414201183431 and Undersampling of 0.9932813049736127


In [42]:
print(f'2.Naive Random Sampling, SMOTE Oversampling and Comnination Sampling (SMOTEENN) have the same recall score with high_risk of 1.0. low_risk of 0.99 and avg/total is 0.99, which is the highest compared to others.')


2.Naive Random Sampling, SMOTE Oversampling and Comnination Sampling (SMOTEENN) have the same recall score with high_risk of 1.0. low_risk of 0.99 and avg/total is 0.99, which is the highest compared to others.


In [43]:
print(f'All models has the same geometric mean score of 0.99')


All models has the same geometric mean score of 0.99
